### References
* https://www.kaggle.com/realtimshady/2lgbm-2nn
* https://www.kaggle.com/munumbutt/feature-engineering-tuned-xgboost-lgbm

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from IPython.core.display import display, HTML
import gc
import plotly.graph_objects as go
from joblib import Parallel, delayed
from sklearn import preprocessing, model_selection
import seaborn as sns
from tqdm import tqdm
from scipy.stats import probplot

pd.set_option('max_rows', 400)
pd.set_option('max_columns', 400)

In [2]:
data_dir = '../input/optiver-realized-volatility-prediction/'

## EDA

In [3]:
sample = pd.read_csv("../input/optiver-realized-volatility-prediction/sample_submission.csv")
sample

,row_id,target
0,0-4,0.003048
1,0-32,0.003048
2,0-34,0.003048


In [4]:
test = pd.read_csv("../input/optiver-realized-volatility-prediction/test.csv")
test

,stock_id,time_id,row_id
0,0,4,0-4
1,0,32,0-32
2,0,34,0-34


In [5]:
train = pd.read_csv("../input/optiver-realized-volatility-prediction/train.csv")
train

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747
...,...,...,...
428927,126,32751,0.003461
428928,126,32753,0.003113
428929,126,32758,0.004070
428930,126,32763,0.003357


In [6]:
train['stock_id'].value_counts()

0      3830
81     3830
94     3830
93     3830
90     3830
89     3830
88     3830
87     3830
86     3830
85     3830
84     3830
83     3830
82     3830
78     3830
62     3830
77     3830
76     3830
74     3830
73     3830
72     3830
70     3830
69     3830
68     3830
67     3830
66     3830
64     3830
95     3830
96     3830
97     3830
98     3830
125    3830
124    3830
123    3830
122    3830
120    3830
119    3830
118    3830
116    3830
115    3830
114    3830
113    3830
112    3830
111    3830
110    3830
109    3830
108    3830
107    3830
105    3830
104    3830
103    3830
102    3830
101    3830
99     3830
1      3830
63     3830
61     3830
16     3830
29     3830
28     3830
27     3830
26     3830
23     3830
22     3830
21     3830
20     3830
19     3830
18     3830
17     3830
15     3830
60     3830
14     3830
11     3830
10     3830
9      3830
8      3830
7      3830
6      3830
5      3830
4      3830
3      3830
2      3830
30     3830
31     3830
32  

In [7]:
train['stock_id'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  26,  27,  28,
        29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        42,  43,  44,  46,  47,  48,  50,  51,  52,  53,  55,  56,  58,
        59,  60,  61,  62,  63,  64,  66,  67,  68,  69,  70,  72,  73,
        74,  75,  76,  77,  78,  80,  81,  82,  83,  84,  85,  86,  87,
        88,  89,  90,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       118, 119, 120, 122, 123, 124, 125, 126])

In [8]:
book_train = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=10')
book_train

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.000127,1.000464,0.999873,1.000548,1,536,100,400
1,5,1,0.999873,1.000127,0.999705,1.000295,100,100,68,10
2,5,2,1.000127,1.000464,1.000042,1.000548,100,636,136,400
3,5,3,0.999705,1.000464,0.999198,1.000548,68,636,100,400
4,5,4,0.999705,1.000380,0.999198,1.000464,68,100,100,600
...,...,...,...,...,...,...,...,...,...,...
2103918,32767,595,0.998724,0.999043,0.998564,0.999202,378,500,584,882
2103919,32767,596,0.998883,0.999043,0.998724,0.999202,400,557,878,882
2103920,32767,597,0.998724,0.998883,0.998564,0.999043,900,200,584,802
2103921,32767,598,0.998724,0.998883,0.998564,0.999043,800,100,584,802


In [9]:
trade_example = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=1")
trade_example

,time_id,seconds_in_bucket,price,size,order_count
0,5,28,1.002080,553,11
1,5,39,1.002460,8,3
2,5,42,1.002308,147,4
3,5,44,1.002788,1,1
4,5,51,1.002657,100,2
...,...,...,...,...,...
296205,32767,579,0.999010,81,3
296206,32767,587,0.999109,50,1
296207,32767,588,0.999010,126,2
296208,32767,592,0.999109,1,1


## FUNC

In [10]:
def convert_to_32bit(df):
    for f in df.columns:
        if df[f].dtype == 'int64':
            df[f] = df[f].astype('int32')
        if df[f].dtype == 'float64':
            df[f] = df[f].astype('float32')
    return df

In [11]:
def wap_1(df):
    wap = (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1'])/(df['bid_size1'] + df['ask_size1'])
    return wap
def wap_2(df):
    wap = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2'])/(df['bid_size2'] + df['ask_size2'])
    return wap
def wap_bid(df):
    wap = (df['bid_price1'] * df['bid_size1'] + df['bid_price2'] * df['bid_size2'])/(df['bid_size1'] + df['bid_size2'])
    return wap
def wap_ask(df):
    wap = (df['ask_price1'] * df['ask_size1'] + df['ask_price2'] * df['ask_size2'])/(df['ask_size1'] + df['ask_size2'])
    return wap

In [12]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))
def count_unique(series):
    return len(np.unique(series))

In [13]:
# Function to read our base train and test set
def read_train_test():
    train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    return train, test

In [14]:
def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df = convert_to_32bit(df)
    
    #calculate return etc
    df['wap1'] = wap_1(df)
    df['log_return1'] = df.groupby('time_id')['wap1'].apply(log_return)
    
    df['wap2'] = wap_2(df)
    df['log_return2'] = df.groupby('time_id')['wap2'].apply(log_return)
    
    df['wap_bid'] = wap_bid(df)
    df['wap_ask'] = wap_ask(df)
    
    df['log_return_bid'] = df.groupby('time_id')['wap_bid'].apply(log_return)
    df['log_return_ask'] = df.groupby('time_id')['wap_ask'].apply(log_return)
    
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1'])/2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    df['bid_volume'] = df['bid_size1'] + df['bid_size2']
    df['ask_volume'] = df['ask_size1'] + df['ask_size2']
    df['bid_ask_volume'] = abs(df['bid_volume'] - df['ask_volume'])
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))

    #dict for aggregate
    create_feature_dict = {
        'wap1': [np.sum, np.mean, np.min, np.max, np.std],
        'wap2': [np.sum, np.mean, np.min, np.max, np.std],
        'log_return1': [np.sum, realized_volatility, np.mean, np.min, np.max, np.std],
        'log_return2': [np.sum, realized_volatility, np.mean, np.min, np.max, np.std],
        'wap_bid': [np.sum, np.mean, np.min, np.max, np.std],
        'wap_ask': [np.sum, np.mean, np.min, np.max, np.std],
        'log_return_bid': [np.sum, realized_volatility, np.mean, np.min, np.max, np.std],
        'log_return_ask': [np.sum, realized_volatility, np.mean, np.min, np.max, np.std],
        'wap_balance': [np.sum, np.mean, np.min, np.max, np.std],
        'price_spread':[np.sum, np.mean, np.min, np.max, np.std],
        'bid_spread':[np.sum, np.mean, np.min, np.max, np.std],
        'ask_spread':[np.sum, np.mean, np.min, np.max, np.std],
        'bid_ask_spread':[np.sum, np.mean, np.min, np.max, np.std],
        'bid_volume':[np.sum, np.mean, np.min, np.max, np.std],
        'ask_volume':[np.sum, np.mean, np.min, np.max, np.std],
        'bid_ask_volume':[np.sum, np.mean, np.min, np.max, np.std],
        'total_volume':[np.sum, np.mean, np.min, np.max, np.std],
        'volume_imbalance':[np.sum, np.mean, np.min, np.max, np.std]
        }

    create_feature_dict_time = {
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility],
        'log_return_bid': [realized_volatility],
        'log_return_bid': [realized_volatility],
    }
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)

    # Merge all
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id__100'], axis = 1, inplace = True)
    
    
    # Create row_id so we can merge
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature

In [15]:
%%time

# Function to preprocess trade data (for each stock id)
def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df = convert_to_32bit(df)
    
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    df['amount']=df['price']*df['size']
    # Dict for aggregations
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, np.max, np.min],
        'order_count':[np.sum,np.max],
        'amount':[np.sum,np.max,np.min],
    }
    create_feature_dict_time = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.sum],
    }
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    

    # Get the stats for different windows
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)
    
    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff/price[1:])*100
        power = np.sum(val*vol[1:])
        return(power)
    
    lis = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]        
        tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)
        # new
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
        energy = np.mean(df_id['price'].values**2)
        iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
        
        # vol vars
        
        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values**2)
        iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)
        
        lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                   'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})
    
    df_lr = pd.DataFrame(lis)
        
   
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
    # Merge all
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id','time_id__100'], axis = 1, inplace = True)
    
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.11 µs


In [16]:
%%time

def get_time_stock(df):
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_400', 'log_return2_realized_volatility_400', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_200', 'log_return2_realized_volatility_200', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_400', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_200']


    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the stock id
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.63 µs


In [17]:
# Funtion to make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, is_train = True):
    
    # Parrallel for loop
    def for_joblib(stock_id):
        # Train
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        # Preprocess book and trade data and merge them
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
        # Return the merge dataframe
        return df_tmp
    
    # Use parallel api to call paralle for loop
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    # Concatenate all the dataframes that return from Parallel
    df = pd.concat(df, ignore_index = True)
    return df

In [18]:
# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

In [19]:
# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False


In [20]:
train, test = read_train_test()

Our training set has 428932 rows


In [21]:
train_stock_ids = train['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
train_ = preprocessor(train_stock_ids, is_train = True)
train = train.merge(train_, on = ['row_id'], how = 'left')

# Get unique stock ids 
test_stock_ids = test['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')

# Get group stats of time_id and stock_id
train = get_time_stock(train)
test = get_time_stock(test)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 40.2min
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed: 97.7min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.4s finished


In [22]:
# replace by order sum (tau)
train['size_tau'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique'] )
test['size_tau'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique'] )
#train['size_tau_450'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_450'] )
#test['size_tau_450'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_450'] )
train['size_tau_400'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_400'] )
test['size_tau_400'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_400'] )
train['size_tau_300'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_300'] )
test['size_tau_300'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_300'] )
#train['size_tau_150'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_150'] )
#test['size_tau_150'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_150'] )
train['size_tau_200'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_200'] )
test['size_tau_200'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_200'] )

In [23]:
train['size_tau2'] = np.sqrt( 1/ train['trade_order_count_sum'] )
test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )
#train['size_tau2_450'] = np.sqrt( 0.25/ train['trade_order_count_sum'] )
#test['size_tau2_450'] = np.sqrt( 0.25/ test['trade_order_count_sum'] )
train['size_tau2_400'] = np.sqrt( 0.33/ train['trade_order_count_sum'] )
test['size_tau2_400'] = np.sqrt( 0.33/ test['trade_order_count_sum'] )
train['size_tau2_300'] = np.sqrt( 0.5/ train['trade_order_count_sum'] )
test['size_tau2_300'] = np.sqrt( 0.5/ test['trade_order_count_sum'] )
#train['size_tau2_150'] = np.sqrt( 0.75/ train['trade_order_count_sum'] )
#test['size_tau2_150'] = np.sqrt( 0.75/ test['trade_order_count_sum'] )
train['size_tau2_200'] = np.sqrt( 0.66/ train['trade_order_count_sum'] )
test['size_tau2_200'] = np.sqrt( 0.66/ test['trade_order_count_sum'] )

# delta tau
train['size_tau2_d'] = train['size_tau2_400'] - train['size_tau2']
test['size_tau2_d'] = test['size_tau2_400'] - test['size_tau2']

In [24]:
df = train
test_data_set = test

In [25]:
test_data_set['stock_id'] = test_data_set['stock_id'].astype(int)
test_data_set.head()

,stock_id,time_id,row_id,wap1_sum,wap1_mean,wap1_amin,wap1_amax,wap1_std,wap2_sum,wap2_mean,wap2_amin,wap2_amax,wap2_std,log_return1_sum,log_return1_realized_volatility,log_return1_mean,log_return1_amin,log_return1_amax,log_return1_std,log_return2_sum,log_return2_realized_volatility,log_return2_mean,log_return2_amin,log_return2_amax,log_return2_std,wap_bid_sum,wap_bid_mean,wap_bid_amin,wap_bid_amax,wap_bid_std,wap_ask_sum,wap_ask_mean,wap_ask_amin,wap_ask_amax,wap_ask_std,log_return_bid_sum,log_return_bid_realized_volatility,log_return_bid_mean,log_return_bid_amin,log_return_bid_amax,log_return_bid_std,log_return_ask_sum,log_return_ask_realized_volatility,log_return_ask_mean,log_return_ask_amin,log_return_ask_amax,log_return_ask_std,wap_balance_sum,wap_balance_mean,wap_balance_amin,wap_balance_amax,wap_balance_std,price_spread_sum,price_spread_mean,price_spread_amin,price_spread_amax,price_spread_std,bid_spread_sum,bid_spread_mean,bid_spread_amin,bid_spread_amax,bid_spread_std,ask_spread_sum,ask_spread_mean,ask_spread_amin,ask_spread_amax,ask_spread_std,bid_ask_spread_sum,bid_ask_spread_mean,bid_ask_spread_amin,bid_ask_spread_amax,bid_ask_spread_std,bid_volume_sum,bid_volume_mean,bid_volume_amin,bid_volume_amax,bid_volume_std,ask_volume_sum,ask_volume_mean,ask_volume_amin,ask_volume_amax,ask_volume_std,bid_ask_volume_sum,bid_ask_volume_mean,bid_ask_volume_amin,bid_ask_volume_amax,bid_ask_volume_std,total_volume_sum,total_volume_mean,total_volume_amin,total_volume_amax,total_volume_std,volume_imbalance_sum,volume_imbalance_mean,volume_imbalance_amin,volume_imbalance_amax,volume_imbalance_std,log_return1_realized_volatility_500,log_return2_realized_volatility_500,log_return_bid_realized_volatility_500,log_return1_realized_volatility_400,log_return2_realized_volatility_400,log_return_bid_realized_volatility_400,log_return1_realized_volatility_300,log_return2_realized_volatility_300,log_return_bid_realized_volatility_300,log_return1_realized_volatility_200,log_return2_realized_volatility_200,log_return_bid_realized_volatility_200,log_return1_realized_volatility_100,log_return2_realized_volatility_100,log_return_bid_realized_volatility_100,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_size_amax,trade_size_amin,trade_order_count_sum,trade_order_count_amax,trade_amount_sum,trade_amount_amax,trade_amount_amin,trade_tendency,trade_f_max,trade_f_min,trade_df_max,trade_df_min,trade_abs_diff,trade_energy,trade_iqr_p,trade_abs_diff_v,trade_energy_v,trade_iqr_p_v,trade_log_return_realized_volatility_500,trade_seconds_in_bucket_count_unique_500,trade_size_sum_500,trade_order_count_sum_500,trade_log_return_realized_volatility_400,trade_seconds_in_bucket_count_unique_400,trade_size_sum_400,trade_order_count_sum_400,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_sum_300,trade_log_return_realized_volatility_200,trade_seconds_in_bucket_count_unique_200,trade_size_sum_200,trade_order_count_sum_200,trade_log_return_realized_volatility_100,trade_seconds_in_bucket_count_unique_100,trade_size_sum_100,trade_order_count_sum_100,log_return1_realized_volatility_mean_stock,log_return1_realized_volatility_std_stock,log_return1_realized_volatility_max_stock,log_return1_realized_volatility_min_stock,log_return2_realized_volatility_mean_stock,log_return2_realized_volatility_std_stock,log_return2_realized_volatility_max_stock,log_return2_realized_volatility_min_stock,log_return1_realized_volatility_400_mean_stock,log_return1_realized_volatility_400_std_stock,log_return1_realized_volatility_400_max_stock,log_return1_realized_volatility_400_min_stock,log_return2_realized_volatility_400_mean_stock,log_return2_realized_volatility_400_std_stock,log_return2_realized_volatility_400_max_stock,log_return2_realized_volatility_400_min_stock,log_return1_realized_volatility_300_mean_stock,log_return1_realized_volatility_300_std_stock,log_return1_realized_

In [26]:
X = df.drop(['row_id', 'target', 'time_id'], axis = 1)
y = df['target']
X.shape, y.shape

((428932, 256), (428932,))

In [27]:
thresh = int(len(df) * 0.9 / df['stock_id'].nunique())
print (thresh)

3446


In [28]:
mask  = df.groupby('stock_id')['stock_id'].cumcount() < thresh

In [29]:
train = df[mask]
test = df[~mask]

In [30]:
X_train = train.drop(['row_id', 'target', 'time_id'], axis = 1)
y_train = train['target']
X_train.shape, y_train.shape

((385952, 256), (385952,))

In [31]:
X_valid = test.drop(['row_id', 'target', 'time_id'], axis = 1)
y_valid = test['target']
X_valid.shape, y_valid.shape

((42980, 256), (42980,))

In [32]:
X_train['stock_id'] = X_train['stock_id'].astype(int)
X_valid['stock_id'] = X_valid['stock_id'].astype(int)

## XGBOOST

In [33]:
import optuna
import xgboost as xgb
from optuna.samplers import TPESampler
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [34]:
params_xgb = {
        'lambda': 0.0014832052084105417, 
        'alpha': 2.6885464964958112, 
        'max_depth': 17, 
        'learning_rate': 0.02, 
        'random_state': 24, 
        'n_estimators': 1540, 
        'eta': 0.12558915915760901, 
        'subsample': 0.6000000000000001, 
        'colsample_bytree': 0.3, 
        'min_child_weight': 77, 
        'reg_lambda': 0.001217091110648466, 
        'reg_alpha': 0.0019723477880301235
        }

xgb_model = xgb.XGBRegressor(**params_xgb, tree_method='gpu_hist')

In [35]:
%%time
xgb_model.fit(X_train ,y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=150, verbose=False)

preds = xgb_model.predict(X_valid)
RMSPE = round(rmspe(y_true = y_valid, y_pred = preds), 5)
print(f'Performance of the Tuned XGB prediction: RMSPE: {RMSPE}')

Performance of the Tuned XGB prediction: RMSPE: 0.25639
CPU times: user 1min 22s, sys: 1.48 s, total: 1min 23s
Wall time: 1min 20s


## LGBM

In [36]:
from lightgbm import LGBMRegressor

In [37]:
params_lgbm = {
        "metric": "rmse",
        "verbosity": -1,
        'learning_rate': 0.04412162462604988, 
        'max_depth': 300, 
        'lambda_l1': 0.12309589568066824, 
        'lambda_l2': 3.1044658548129586e-06, 
        'num_leaves': 246, 
        'n_estimators': 2350, 
        'feature_fraction': 0.531654883966269, 
        'bagging_fraction': 0.8553165643797457, 
        'bagging_freq': 8, 
        'min_child_samples': 42
        }

In [38]:
lgbm_model = LGBMRegressor(**params_lgbm, device='gpu')

In [39]:
%%time
lgbm_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False, early_stopping_rounds=150)

preds = lgbm_model.predict(X_valid)
RMSPE = round(rmspe(y_true = y_valid, y_pred = preds), 5)
print(f'Performance of the Tuned LIGHTGBM prediction: RMSPE: {RMSPE}')

[LightGBM] [Warning] bagging_fraction is set=0.8553165643797457, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8553165643797457
[LightGBM] [Warning] lambda_l1 is set=0.12309589568066824, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.12309589568066824
[LightGBM] [Warning] feature_fraction is set=0.531654883966269, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.531654883966269
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] lambda_l2 is set=3.1044658548129586e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.1044658548129586e-06
Performance of the Tuned LIGHTGBM prediction: RMSPE: 0.25678
CPU times: user 7min 15s, sys: 5.9 s, total: 7min 21s
Wall time: 3min 57s


## CatBoostRegressor

In [40]:
import catboost as cat
from catboost import CatBoostRegressor

In [41]:
params_cb = {
        'colsample_bylevel': 0.029576065862676762,
        'depth': 91,
        'learning_rate': 0.022293479743970765,
        'iterations': 7000,
        'max_bin': 120,
        'min_data_in_leaf': 66,
        'l2_leaf_reg': 0.0009704826955054485,
        'bagging_temperature': 0.7432417203968587,
        'subsample': 0.7022796507235656,
        'grow_policy': 'Lossguide', 
        'leaf_estimation_method': 'Newton',
        'loss_function': 'RMSE',
        'eval_metric': 'RMSE',
        'cat_features': ['stock_id']
        }

In [42]:
cb_model = CatBoostRegressor(**params_cb)

In [43]:
%%time
cb_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False, early_stopping_rounds=150)

preds = cb_model.predict(X_valid)
RMSPE = round(rmspe(y_true = y_valid, y_pred = preds), 5)
print(f'Performance of the Tuned CATBOOST prediction: RMSPE: {RMSPE}')

Performance of the Tuned CATBOOST prediction: RMSPE: 0.25953
CPU times: user 38min 17s, sys: 56.6 s, total: 39min 14s
Wall time: 21min 17s


## Stacking

In [44]:
from sklearn.ensemble import StackingRegressor

In [45]:
mod_xgb = xgb.XGBRegressor(tree_method='gpu_hist', n_jobs= - 1)
mod_lgbm = LGBMRegressor(device='gpu')
mod_cb = CatBoostRegressor()

In [46]:
estimators = [('mod_xgb', mod_xgb),
              ('mod_lgbm', mod_lgbm),
              ('mod_cb', mod_cb)]

clf = StackingRegressor(estimators=estimators, verbose=1)

In [47]:
%%time
clf.fit(X_train, y_train)

Learning rate set to 0.112669
0:	learn: 0.0027038	total: 475ms	remaining: 7m 54s
1:	learn: 0.0024937	total: 1.19s	remaining: 9m 55s
2:	learn: 0.0023114	total: 1.89s	remaining: 10m 27s
3:	learn: 0.0021547	total: 2.32s	remaining: 9m 37s
4:	learn: 0.0020185	total: 2.73s	remaining: 9m 4s
5:	learn: 0.0019030	total: 3.36s	remaining: 9m 16s
6:	learn: 0.0018023	total: 3.91s	remaining: 9m 14s
7:	learn: 0.0017159	total: 4.26s	remaining: 8m 48s
8:	learn: 0.0016423	total: 4.67s	remaining: 8m 34s
9:	learn: 0.0015795	total: 5.08s	remaining: 8m 23s
10:	learn: 0.0015268	total: 5.48s	remaining: 8m 12s
11:	learn: 0.0014819	total: 5.8s	remaining: 7m 57s
12:	learn: 0.0014450	total: 6.16s	remaining: 7m 47s
13:	learn: 0.0014126	total: 6.61s	remaining: 7m 45s
14:	learn: 0.0013860	total: 6.98s	remaining: 7m 38s
15:	learn: 0.0013612	total: 7.37s	remaining: 7m 33s
16:	learn: 0.0013418	total: 7.7s	remaining: 7m 25s
17:	learn: 0.0013252	total: 8.02s	remaining: 7m 17s
18:	learn: 0.0013101	total: 8.4s	remaining: 7m

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   24.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.9min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Learning rate set to 0.108281
0:	learn: 0.0026488	total: 477ms	remaining: 7m 56s
1:	learn: 0.0024512	total: 1.03s	remaining: 8m 34s
2:	learn: 0.0022789	total: 1.64s	remaining: 9m 5s
3:	learn: 0.0021295	total: 1.99s	remaining: 8m 14s
4:	learn: 0.0019998	total: 2.27s	remaining: 7m 32s
5:	learn: 0.0018881	total: 2.58s	remaining: 7m 8s
6:	learn: 0.0017889	total: 2.9s	remaining: 6m 51s
7:	learn: 0.0017043	total: 3.22s	remaining: 6m 39s
8:	learn: 0.0016298	total: 3.52s	remaining: 6m 27s
9:	learn: 0.0015672	total: 4.02s	remaining: 6m 37s
10:	learn: 0.0015139	total: 4.45s	remaining: 6m 40s
11:	learn: 0.0014688	total: 4.73s	remaining: 6m 29s
12:	learn: 0.0014300	total: 5.08s	remaining: 6m 25s
13:	learn: 0.0013971	total: 5.38s	remaining: 6m 18s
14:	learn: 0.0013689	total: 5.72s	remaining: 6m 15s
15:	learn: 0.0013449	total: 6.01s	remaining: 6m 9s
16:	learn: 0.0013238	total: 6.36s	remaining: 6m 7s
17:	learn: 0.0013058	total: 6.66s	remaining: 6m 3s
18:	learn: 0.0012908	total: 6.93s	remaining: 5m 57

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 27.8min finished


CPU times: user 1h 6min 21s, sys: 31.9 s, total: 1h 6min 53s
Wall time: 37min 10s


StackingRegressor(estimators=[('mod_xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=-1,
                                            num_parallel_tree=None,
                                            random_state=None, reg_alpha=None

In [48]:
preds = clf.predict(X_valid)
RMSPE = round(rmspe(y_true = y_valid, y_pred = preds), 5)
print(f'Performance of the STACK prediction: RMSPE: {RMSPE}')

Performance of the STACK prediction: RMSPE: 0.25643


## Submission

In [49]:
y_pred = test_data_set[['row_id']]
X_test = test_data_set.drop(['time_id', 'row_id'], axis = 1)

In [50]:
target = np.zeros(len(X_test))

pred = clf.predict(X_test[X_train.columns])
target = pred

In [51]:
y_pred = y_pred.assign(target = target)
y_pred

,row_id,target
0,0-4,0.006828
1,0-32,0.008585
2,0-34,0.008585


In [52]:
y_pred.to_csv('submission.csv',index = False)